In [10]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import math

In [3]:
data = pd.read_csv('train.csv',sep=',',header=0)

In [4]:
data

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13
0,22.6,0.06724,0.0,3.24,0.0,0.460,6.333,17.2,5.2146,4.0,430.0,16.9,375.21,7.34
1,50.0,9.23230,0.0,18.10,0.0,0.631,6.216,100.0,1.1691,24.0,666.0,20.2,366.15,9.53
2,23.0,0.11425,0.0,13.89,1.0,0.550,6.373,92.4,3.3633,5.0,276.0,16.4,393.74,10.50
3,8.3,24.80170,0.0,18.10,0.0,0.693,5.349,96.0,1.7028,24.0,666.0,20.2,396.90,19.77
4,21.2,0.05646,0.0,12.83,0.0,0.437,6.232,53.7,5.0141,5.0,398.0,18.7,386.40,12.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,16.6,0.67191,0.0,8.14,0.0,0.538,5.813,90.3,4.6820,4.0,307.0,21.0,376.88,14.81
146,32.9,0.01778,95.0,1.47,0.0,0.403,7.135,13.9,7.6534,3.0,402.0,17.0,384.30,4.45
147,36.2,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
148,11.0,7.36711,0.0,18.10,0.0,0.679,6.193,78.1,1.9356,24.0,666.0,20.2,96.73,21.52


In [5]:
y = (data['y']).to_numpy()

In [6]:
X = (data.iloc[:,1:]).to_numpy()

In [7]:
X

array([[6.72400e-02, 0.00000e+00, 3.24000e+00, ..., 1.69000e+01,
        3.75210e+02, 7.34000e+00],
       [9.23230e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.66150e+02, 9.53000e+00],
       [1.14250e-01, 0.00000e+00, 1.38900e+01, ..., 1.64000e+01,
        3.93740e+02, 1.05000e+01],
       ...,
       [6.90500e-02, 0.00000e+00, 2.18000e+00, ..., 1.87000e+01,
        3.96900e+02, 5.33000e+00],
       [7.36711e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        9.67300e+01, 2.15200e+01],
       [1.68118e+01, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.96900e+02, 3.08100e+01]])

In [17]:
kf = KFold(n_splits=10)
results = []
for l in [0.1,1,10,100,200]:
    ridge = linear_model.Ridge(alpha=l)
    split_results = []
    for train_index,test_index in kf.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        ridge.fit(X_train,y_train)
        y_pred = ridge.predict(X_test)
        split_results.append(math.sqrt(mean_squared_error(y_test,y_pred)))
    print("msqs: " + str(split_results))
    results.append(sum(split_results)/10)
print("averaged errors: " + str(results))
f = open('sample.csv','w')
for x in results:
    f.write(str(x)+'\n')
f.close()





msqs: [7.339081403942325, 4.545067284123984, 7.549093338326529, 5.423093816937797, 3.9815936113697625, 5.893819123664637, 6.402006051497151, 5.656822166712559, 4.62334568880179, 3.604171965202057]
msqs: [7.46516296597251, 4.432380508549532, 7.553463335611648, 5.3007461805953815, 3.9652211893189397, 5.828557280742762, 6.467339124587824, 5.839534869453775, 4.716844979762119, 3.4291369781865018]
msqs: [7.642463915950154, 3.8105547873462333, 7.515255071872815, 5.326231864693384, 4.042384584875126, 6.094279726278974, 6.44799272553288, 6.048903453504453, 4.607805063407087, 3.3004436672617823]
msqs: [8.138291150730687, 3.0668399030900892, 7.339536153314462, 5.719732141364704, 4.2721413291989965, 7.222153591982238, 6.492213659605742, 6.561522159989143, 4.129432297780134, 3.424558967084151]
msqs: [8.35811773692188, 3.080817843898126, 7.345319575217206, 5.689633235568394, 4.361321413702476, 7.620629689515378, 6.551950726900713, 6.719535289219414, 3.9723601567672318, 3.512651530900441]
averaged e

In [ ]:
kf = KFold(n_splits=10)
results = []
for l in [0.1,1,10,100,200]:
    ridge = linear_model.RidgeCV(alphas=[l],cv=kf)
    split_results = []
    for train_index,test_index in kf.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        ridge.fit(X_train,y_train)
        y_pred = ridge.predict(X_test)
        split_results.append(math.sqrt(mean_squared_error(y_test,y_pred)))
    print("msqs: " + str(split_results))
    results.append(sum(split_results)/10)
print("averaged errors: " + str(results))
f = open('sample.csv','w')
for x in results:
    f.write(str(x)+'\n')
f.close()
